## Import libaries

In [1]:
import os
import sys
assert os.path.exists('../src'), f"[ERROR] The path src not detected in the parent directory '{os.getcwd()}'."

if os.getcwd().endswith('/notebooks'):
    os.chdir('..')
    sys.path.append('./src')

print(f'[INFO] Current Directory: "{os.getcwd()}".')

[INFO] Current Directory: "/home/iceking/Desktop/22SS-TUM Lecture Docs/3. Praktikum (IN2106, IN4249)/Repo/topic-modeling-advancements".


In [2]:
%load_ext autoreload
%autoreload 2


from src import preprocessor
from src.utils import load_documents
import nltk
import json
import ast
import pandas as pd
import time
import shutil

In [3]:
from src.bulk_runner import OUTPUT_FOLDER
from src.bulk_runner import main_runner

2022-07-24 10:25:35.819315: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-24 10:25:35.819342: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
if os.path.exists(OUTPUT_FOLDER):
    shutil.rmtree(OUTPUT_FOLDER)

In [20]:
def delete_first_line(filename:str,done_file:str):
    with open(filename, 'r') as fin:
        data = fin.read().splitlines(True)
    with open(done_file, 'a+') as fout:
        fout.write(data[0])
    with open(filename, 'w') as fout:
        fout.writelines(data[1:])

In [21]:
combinations = []

for comb_filename in ['combinations.txt','combinations_ctm.txt']:
    with open(comb_filename) as comb_file:
        for i,item in enumerate(comb_file.readlines()):
            #item = item.replace("'",'"')
            x= ast.literal_eval(item)
            combinations.append((x,comb_filename))

In [22]:
dfs_merged=pd.DataFrame()
for i,(configs,configs_filename) in enumerate(combinations):
    print(f'[INFO] Configs #{i}/{len(combinations)}: {configs}')
    try:
        delete_first_line(configs_filename,configs_filename + '_done')
        df_output_topic_word = main_runner(configs=configs)
        dfs_merged = pd.concat([dfs_merged,df_output_topic_word])
        dfs_merged.to_csv(f'./{OUTPUT_FOLDER}/merged.csv')
    except Exception:
        print('[WARN] Current execution gave an error!')
        time.sleep(10)


[INFO] Configs #0/1961: {'algorithm_args': {'embedding_model': 'all-distilroberta-v1', 'top_n_words': 10, 'algorithm': 'bertopic', 'n_gram_range_tuple': (1, 1), 'min_docs_per_topic': 15, 'num_topics': 4, 'cluster_model': 'hdbscan', 'hdbscan_args': {'min_cluster_size': 15, 'metric': 'euclidean', 'cluster_selection_method': 'eom', 'prediction_data': True}, 'umap_args': {'n_neighbors': 15, 'n_components': 3, 'min_dist': 0.0, 'metric': 'cosine', 'low_memory': False, 'random_state': 42}}, 'preprocessing_funcs': ['to_lowercase', 'standardize_accented_chars', 'remove_url', 'expand_contractions', 'remove_mentions', 'remove_hashtags', 'keep_only_alphabet', 'remove_english_stop_words', 'lemmatize_noun'], 'dataset': 'crisis_01'}
[INFO] [PREPROCESSOR] Available Preprocessing Functions in the Module:['to_lowercase', 'standardize_accented_chars', 'remove_url', 'expand_missing_delimiter', 'remove_mentions', 'remove_hashtags', 'keep_only_alphabet', 'remove_new_lines', 'remove_extra_spaces', 'remove_ht

Batches:   0%|          | 0/642 [00:00<?, ?it/s]

KeyboardInterrupt: 